## Lab 1

### Part 1

### Import Libraries

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import random

In [2]:
#attribute url
url = "https://www.popvortex.com/music/charts/top-100-songs.php"

In [3]:
#get request
response = requests.get(url)
response.status_code

200

In [4]:
soup = BeautifulSoup(response.content, "html.parser")

In [5]:
#divide the different criteria into a different lists to be put into our final dataframe

artist = []
song = []
genre = []
year = []

num_iter = len("body > div.container > div:nth-child(4) > div.col-xs-12.col-md-8 > div.chart-wrapper > div.feed-item")

songart = soup.select("body > div.container > div:nth-child(4) > div.col-xs-12.col-md-8 > div.chart-wrapper > div.feed-item")
genlist = soup.select("body > div.container > div:nth-child(4) > div.col-xs-12.col-md-8 > div.chart-wrapper > div.feed-item > div.chart-content")
yearlist = soup.select("body > div.container > div:nth-child(4) > div.col-xs-12.col-md-8 > div.chart-wrapper > div.feed-item > div.chart-content > ul > li:nth-child(2)")

for i in range(num_iter):
    artist.append(songart[i].em.get_text())
    song.append(songart[i].cite.get_text())
    year.append(yearlist[i].get_text())    
    try:
        genre.append(genlist[i].ul.li.a.get_text())
    except:
        genre.append('Unknown')

    

In [6]:
#Attribute the lists to a dataframe

top100 = pd.DataFrame({'artist':artist
                    ,'track':song
                    ,'genre':genre
                    ,'year':year})

In [7]:
top100

,artist,track,genre,year
0,Lizzo,About Damn Time,Pop,"Release Date: April 14, 2022"
1,Kate Bush,Running Up That Hill (A Deal with God),Pop,"Release Date: August 5, 1985"
2,P!nk,Irrelevant,New Release,Genre: Pop
3,Walker Hayes,Y'all Life,New Release,Genre: Country
4,Tom MacDonald,Names,New Release,Genre: Hip-Hop / Rap
...,...,...,...,...
95,j-hope,STOP,New Release,Genre: K-Pop
96,Yung Gravy,Betty (Get Money),Hip-Hop / Rap,"Release Date: June 10, 2022"
97,Glass Animals,Heat Waves,Alternative,"Release Date: June 29, 2020"
98,One Direction,Night Changes,Pop,"Release Date: November 17, 2014"


In [8]:
#In the year column we have a Genre: leak. We will split these genres...  

def genresplit(value):
    if 'Genre:' in value:
        name = value.split('Genre: ')[1]
        
        return name
    else:
        return value

In [9]:
#...move these genres 
top100['genre'] = top100['genre'].apply(genresplit)
top100['year'] = top100['year'].apply(genresplit)

In [10]:
def tidy(value):

    type1 = value['genre']
    type2 = value['year']
    if str(type1) == 'New Release':
        return type2
    else:
        return type1

    
top100['genre'] = top100.apply(tidy,axis=1)

In [11]:
#finally split the year column by the comma and find the year value. "Nan" values filled with 2022, because it was originally a "New Release" value
top100['year'] = top100['year'].str.split(", ", n = 1, expand = True)[1]
top100['year'] = top100['year'].fillna(2022)
top100['track'] = top100['track'].str.strip()

In [12]:
top100

,artist,track,genre,year
0,Lizzo,About Damn Time,Pop,2022
1,Kate Bush,Running Up That Hill (A Deal with God),Pop,1985
2,P!nk,Irrelevant,Pop,2022
3,Walker Hayes,Y'all Life,Country,2022
4,Tom MacDonald,Names,Hip-Hop / Rap,2022
...,...,...,...,...
95,j-hope,STOP,K-Pop,2022
96,Yung Gravy,Betty (Get Money),Hip-Hop / Rap,2022
97,Glass Animals,Heat Waves,Alternative,2020
98,One Direction,Night Changes,Pop,2014


### Part 2

In [13]:
urls = "https://spinditty.com/playlists/100songs"

In [14]:
responses = requests.get(urls)
responses.status_code

200

In [15]:
sopa = BeautifulSoup(responses.content, "html.parser")

In [16]:
##attribute different values to song, artist, genre and year lists. 

songartist2 = []
genre2 = []
years2 = []
num_iter = len("body > div.container > div:nth-child(4) > div.col-xs-12.col-md-8 > div.chart-wrapper > div.feed-item")

songarts = sopa.select("body > phoenix-page > div > div > div.m-page > section > article > div > div > section > div > div > h2")
genres = sopa.select("body > phoenix-page > div > div > div.m-page > section > article > div > div > section > div > div > p")
years = sopa.select("body > phoenix-page > div > div > div.m-page > section > article > div > div > section > div > div > h3")
for i in range(len(songarts)):
    songartist2.append(songarts[i].get_text())
    
for i in range(len(genres)):
    genre2.append(genres[i].get_text())
        
for i in range(len(years)):
    years2.append(years[i].get_text())


In [17]:
##find genre in text. 

genrefin = []
for i in genre2:
    if 'Genre:' in i:
        genrefin.append(i)
    elif 'GENRE:' in i:
        genrefin.append(i)

        
##filter any unneeded rows. 

songartfin = []        
for j in songartist2:
    if '#' in j:
        songartfin.append(j)
        

#treat values longer than years
lap = []
for i in years2:
    if len(i) > 4:
        i = re.findall('\d{4}',i)
        lap.append(i)
    else:
        lap.append(i)
years2 = lap

In [18]:
#drop any duplicates, then create the dataframe for table2 

top100s = pd.DataFrame({'go':songartfin})
top100s = top100s.drop_duplicates().reset_index()
top100s = pd.DataFrame({'go': top100s['go'], 'genre':genrefin, 'year':years2})

In [19]:
top100s

,go,genre,year
0,"#100 - ""Bitter Sweet Symphony"" by The Verve",Genre: Britpop,[1997]
1,"#99: ""For What It's Worth"" by Buffalo Springfield",Genre: Folk Rock,1967
2,"#98 ""Fire and Rain"" by James Taylor",Genre: Folk Rock,[1970]
3,"#97: ""Rolling in the Deep"" by Adele",Genre: Pop,[2011]
4,"#96 ""(We're Gonna) Rock Around the Clock"" by B...",Genre: Rock & Roll,1954
...,...,...,...
95,"#5: ""Stairway to Heaven""",Genre: Rock,[1971]
96,"#4: ""Imagine"" by John Lennon",Genre: Rock,[1971]
97,"#3: ""Like a Rolling Stone"" by Bob Dylan",Genre: Folk Rock,[1965]
98,"#2: ""Yesterday"" by The Beatles",Genre: British Invasion,[1965]


In [20]:
#determine artist, genre and track columns, as well as year cleaning. 

top100s['track'] = top100s['go'].str.split("by", n = 1, expand = True)[0]
top100s['artist'] = top100s['go'].str.split("by", n = 1, expand = True)[1]
top100s['track'] = top100s['track'].str.split('"', n = 1, expand = True)[1]
top100s['genre'] = top100s['genre'].str.split(': ', n = 1, expand = True)[1]
top100s['track'] = top100s['track'].str.replace('"','')

yearconvert = []

#if it's a list:
for i in top100s['year']:
    if isinstance(i,list):
        yearconvert.append(i[0])
    else:
        yearconvert.append(i)

top100s['year'] = yearconvert

top100s['track'] = top100s['track'].str.strip()

In [21]:
top100s = top100s[['track','artist','genre','year']]

In [22]:
top100s

,track,artist,genre,year
0,Bitter Sweet Symphony,The Verve,Britpop,1997
1,For What It's Worth,Buffalo Springfield,Folk Rock,1967
2,Fire and Rain,James Taylor,Folk Rock,1970
3,Rolling in the Deep,Adele,Pop,2011
4,(We're Gonna) Rock Around the Clock,Bill Haley and His Comets,Rock & Roll,1954
...,...,...,...,...
95,Stairway to Heaven,None,Rock,1971
96,Imagine,John Lennon,Rock,1971
97,Like a Rolling Stone,Bob Dylan,Folk Rock,1965
98,Yesterday,The Beatles,British Invasion,1965


In [23]:
#concatenate into one table
top100s = top100s[['track','artist','genre','year']]
top100 = top100[['track','artist','genre','year']]
top = pd.concat([top100,top100s], axis = 0).reset_index()
top = top.drop(columns='index')

In [24]:
top

,track,artist,genre,year
0,About Damn Time,Lizzo,Pop,2022
1,Running Up That Hill (A Deal with God),Kate Bush,Pop,1985
2,Irrelevant,P!nk,Pop,2022
3,Y'all Life,Walker Hayes,Country,2022
4,Names,Tom MacDonald,Hip-Hop / Rap,2022
...,...,...,...,...
195,Stairway to Heaven,None,Rock,1971
196,Imagine,John Lennon,Rock,1971
197,Like a Rolling Stone,Bob Dylan,Folk Rock,1965
198,Yesterday,The Beatles,British Invasion,1965


## Lab 2

In [38]:
def musicrecommender():
    song = input("Name of your song: ")    
    if song in list(top['track']):
        print(top['track'].iloc[random.randint(0,200)])    
    elif song not in list(top['track']):
        print("Sorry, we cannot recommend any songs")

In [39]:
musicrecommender()

Name of your song: Yesterday
Shivers
